# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"

In [11]:
# region of interest is Amazonia
roi = ee.FeatureCollection(f"{data_folder}/raw/biomes_br").filter(ee.Filter.eq("CD_Bioma", 1)).geometry()

# For specific biomes, the feature "CD_Bioma" is:
# 1 = Amazonia
# 2 = Caatinga
# 3 = Cerrado
# 4 = Mata Atlantica
# 5 = Pampa
# 6 = Pantanal

In [7]:
years = 5

if (years == 0):
    land_use_years = ee.Image(f"{data_folder}/land_use")
    name = "all_LULC"
else:
    land_use_years = ee.Image(f"{data_folder}/land_use_{years}_years")
    name = f"{years}y_LULC"

age_agbd = ee.Image(f"{data_folder}/age_agbd")
categorical = ee.Image(f"{data_folder}/categorical")
sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask").rename("one_hectare_mask")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave").float().rename("cwd")
yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
yearly_SI = ee.Image(f"{data_folder}/yearly_SI")

combined_mask = one_hectare_mask.And(land_use_years.select("last_LU"))

unified_data = (
    age_agbd.addBands(
        [categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
    ]).updateMask(combined_mask)
)

unified_data_fc = unified_data.stratifiedSample(
    numPoints = 5000, classBand = 'ecoreg', region = unified_data.geometry()
)

# Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
task = ee.batch.Export.table.toDrive(
    collection=unified_data_fc, description=name
)
task.start()
